![](pictures/shopping_propensity.png)

<b>  </b>  
<b>  </b>  
<b>  </b>  
<b>  </b>  
Ноутбук от 30.09.23 
<b>  </b>  
<b>  </b>  
<b>  </b>  
<b>  </b>  
<b>  </b>  
<b>  </b>  
<b>  </b>   
**_"Data Science Analyst"_  
_МОДЕЛЬ СКЛОННОСТИ КЛИЕНТОВ К ПОКУПКАМ В ФЕДЕРАЛЬНОЙ СЕТИ ПРОДАЖ СПОРТИВНЫХ ТОВАРОВ_**  

Выполнено по технологии CRISP-DM:  
Аргентов Сергей, г.Липецк  
https://argentov.pro
<b>  </b>  
<b>  </b>  

<h1>СОДЕРЖАНИЕ<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-Предобработка-данных" data-toc-modified-id="1.-Предобработка-данных-1">1. Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#1.1.-Анализ-источников-данных" data-toc-modified-id="1.1.-Анализ-источников-данных-1.1">1.1. Анализ источников данных</a></span></li><li><span><a href="#1.2.-Устранение-пропусков-в-данных" data-toc-modified-id="1.2.-Устранение-пропусков-в-данных-1.2">1.2. Устранение пропусков в данных</a></span><ul class="toc-item"><li><span><a href="#1.2.1.-Проверим-гипотезу-что-для-товаров-с-пропуском-в-признаке-&quot;гендерная-принадлежность-потенциальных-покупателей-товара&quot;-нет-признаков-принадлежности-к-полу-в-наименовании-товара" data-toc-modified-id="1.2.1.-Проверим-гипотезу-что-для-товаров-с-пропуском-в-признаке-&quot;гендерная-принадлежность-потенциальных-покупателей-товара&quot;-нет-признаков-принадлежности-к-полу-в-наименовании-товара-1.2.1">1.2.1. Проверим гипотезу что для товаров с пропуском в признаке "гендерная принадлежность потенциальных покупателей товара" нет признаков принадлежности к полу в наименовании товара</a></span></li><li><span><a href="#1.2.2.-Проверим-гипотезу-что-для-товаров-с-пропуском-в-признаке-&quot;цвет&quot;-есть-одноимённые-товары-с-наличием-значения-в-признаке-&quot;цвет&quot;" data-toc-modified-id="1.2.2.-Проверим-гипотезу-что-для-товаров-с-пропуском-в-признаке-&quot;цвет&quot;-есть-одноимённые-товары-с-наличием-значения-в-признаке-&quot;цвет&quot;-1.2.2">1.2.2. Проверим гипотезу что для товаров с пропуском в признаке "цвет" есть одноимённые товары с наличием значения в признаке "цвет"</a></span></li></ul></li><li><span><a href="#1.3.-Обогащение-данных-таблицы-personal_data-из-дополнительного-источника" data-toc-modified-id="1.3.-Обогащение-данных-таблицы-personal_data-из-дополнительного-источника-1.3">1.3. Обогащение данных таблицы personal_data из дополнительного источника</a></span></li></ul></li><li><span><a href="#2.-Бинарная-классификация" data-toc-modified-id="2.-Бинарная-классификация-2">2. Бинарная классификация</a></span></li><li><span><a href="#3.-Анализ-проведённого-АВ-тестирования" data-toc-modified-id="3.-Анализ-проведённого-АВ-тестирования-3">3. Анализ проведённого АВ-тестирования</a></span></li><li><span><a href="#4.-Кластерезация" data-toc-modified-id="4.-Кластерезация-4">4. Кластерезация</a></span></li><li><span><a href="#5.-Построение-модели-склорнности-клиентов-к-покупкам" data-toc-modified-id="5.-Построение-модели-склорнности-клиентов-к-покупкам-5">5. Построение модели склорнности клиентов к покупкам</a></span></li></ul></div>

In [260]:
# Загрузка биьлиотек
import pandas as pd
import sqlite3 as db

import datetime as dt
import multiprocessing
from multiprocessing import Pool, Manager

# 1. Предобработка данных

## 1.1. Анализ источников данных

In [4]:
# Предоставленная база данных
with db.connect('data/result_data/shop_database.db') as con:
    rows = con.cursor().execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
    dfs = {}
    for row in rows:
        dfs[row[0]] = pd.read_sql(f'SELECT * FROM {row[0]}', con)
        print(f'\n{row[0]}')
        print(dfs[row[0]].info())


purchases
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786260 entries, 0 to 786259
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   id           786260 non-null  int64  
 1   product      786260 non-null  object 
 2   colour       666736 non-null  object 
 3   cost         786260 non-null  int64  
 4   product_sex  471548 non-null  float64
 5   base_sale    786260 non-null  int64  
 6   dt           786260 non-null  int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 42.0+ MB
None

personal_data_coeffs
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104989 entries, 0 to 104988
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             104989 non-null  int64  
 1   lbt_coef       104989 non-null  float64
 2   ac_coef        104989 non-null  float64
 3   sm_coef        104989 non-null  float64
 4   personal_coef  10

In [283]:
colors = set(list(dfs['purchases'].colour))
len(colors)

1694

In [288]:
colors_unique = set()
for palette in colors:
    p = palette.split('/')
    print(p)
    colors_unique.union(p)
colors_unique

['черный', 'белый', 'голубой', 'красный']
['темно-коричневый']
['голубой', 'мятный']
['серо-голубой', 'белый', 'оранжевый']
['темно-синий', 'ярко-розовый', 'белый']
['льняной', 'бежевый']
['черный', 'лавандовый', 'бежевый']
['белый', 'графитовый', 'красный']
['голубой', 'зеленый']
['оранжевый', 'зеленый']
['белый', 'синий', 'черный']
['чёрно-белый']
['черный', 'белый', 'мятный']
['желтый', 'черный', 'белый']
['глянцевый']
['оранжевый', 'синий']
['белый', 'синий']
['светло-серый', 'светло-сиреневый']
['темно-серый', 'чёрный']
['белый', 'желтый', 'розовый']
['серый', 'салатовый']
['белый', 'голубой', 'черный']
['бежевый', 'красный', 'белый']
['синий', 'красный', 'оранжевый']
['желтый', 'оранжевый', 'розовый']
['болотный', 'зеленый']
['серый', 'голубой', 'белый']
['клетчатый', 'коралловый']
['оранжевый', 'красный', 'белый']
['Красный']
['черный', 'синий', 'оранжевый']
['голубой', 'сиреневый']
['молочный']
['серый', 'черный', 'синий']
['светло-желтый']
['тёмно-красный']
['белый', 'серый', 

AttributeError: 'NoneType' object has no attribute 'split'

In [163]:
dfs['personal_data_coeffs'].head(3)

,id,lbt_coef,ac_coef,sm_coef,personal_coef
0,0,5.078678,-0.307147,0.959027,0.5072
1,3,7.764766,-0.030225,0.794720,0.4304
2,4,4.569378,0.063693,0.820892,0.5072


In [164]:
dfs['personal_data'].head(3)

,id,gender,age,education,city,country
0,0,0,36,среднее,1201,32
1,4,0,35,среднее,1134,32
2,6,1,52,среднее,1188,32


## 1.2. Устранение пропусков в данных

Пропущенные значения есть в таблице `product_sex` (15%) и `color` (40%). Это может быть следствием технической/организационой ошибок при заполнении данных или может отображать следующее:  
- для колонки `product_sex` в случае унисекс товаров данные о гендерная принадлежность потенциальных покупателей товара не проставлялась,
- для колонки `colour` в случае когда товар не имеет цветовой гаммы (пищевые продукты, информационные материалы и т.п.) цвет товара не проставляется.  
Проверим вы шеуказанные гипотизы

### 1.2.1. Проверим гипотезу что для товаров с пропуском в признаке "гендерная принадлежность потенциальных покупателей товара" нет признаков принадлежности к полу в наименовании товара

In [170]:
# Соберём уникальный перечень покупок
df_product_sex = dfs['purchases'].groupby('product').agg({'product_sex': 'max'})
df_product_sex['product'] = df_product_sex.index
df_product_sex.reset_index(drop=True, inplace=True)
print(f'Уникальные значения гендерной принадлежности '
      f'потенциального покупателя: {df_product_sex.product_sex.unique()}\n'
      f'Количесвто уникальных товаров: {len(df_product_sex)}')

Уникальные значения гендерной принадлежности потенциального покупателя: [2. 0. 1.]
Количесвто уникальных товаров: 23145


In [179]:
# Проверим какие покупки и с какими признаками гендерной принадлежности
# имеют в наименовании товара слова 'муж, жен'
def check_product_name_sex(x, fit) -> int:
    if fit in x:
        return 1
    else:
        return 0

df_product_sex['муж'] = df_product_sex['product'].\
                        apply(lambda x:check_product_name_sex(x,'муж'))
df_product_sex['жен'] = df_product_sex['product'].\
                        apply(lambda x: check_product_name_sex(x,'жен'))

In [178]:
# Проверим все ли товары,
# не имеющие гендерной принадлежности потенциального покупателя
# не имеют в названии явного указания на пол 'муж', 'жен'
df_product_sex['унисекс'] = df_product_sex.\
                            apply(lambda x: 1
                                 if x['муж'] == 0 and x['жен'] == 0
                                 else 0,
                                 axis=1)
# Определим количество стрк где в названиии продукта нет гендерной принадлежности
# потенциального покупатнлы и не проставлены соответсвующие признаки
len(df_product_sex[(df_product_sex['унисекс'] != 1) & (df_product_sex.product_sex.isna())])

0

**Вывод:** В признаке `product_sex` пропущенные данные с большой вероятностью обозначают что товар пренадлежит к категории `унисекс`. Для возможности использования данного признака в дальнейшем проставим ему в датасете значение 2.0

In [173]:
# Заменим пустые значения признака `product_sex` на значение 2.0
dfs['purchases'].product_sex = dfs['purchases'].product_sex.fillna(2.0)

### 1.2.2. Проверим гипотезу что для товаров с пропуском в признаке "цвет" есть одноимённые товары с наличием значения в признаке "цвет"

Для заполнения пропусков в поле colour проверим гипотезу об уникальности данного признака для каждого проданного продукта 

In [249]:
df_colour = dfs['purchases'][['product', 'colour']].\
                        sort_values(['product', 'colour'])
df_colour.head(6)

,product,colour
305558,"8 батареек GP, АА, 8 шт.",зеленый
346983,"8 батареек GP, АА, 8 шт.",зеленый
528044,"8 батареек GP, АА, 8 шт.",зеленый
59533,"8 батареек GP, АА, 8 шт.",None
303537,"Apple Watch Series 8, 41 мм, корпус из алюминия",черный/серый
498024,"Apple Watch Series 8, 41 мм, корпус из алюминия",черный/серый


In [261]:
# Проставим к каждому товару наиболее часто встречающееся значение цвета
# в мультипроцессорном режиме
def freq_product_color(product, df) -> str:
    return df[df['product'] == product].describe()['colour']['top']

def add_freq_product_color(df, df_unique):
    df_unique['freq_color'] = df_unique['product'].\
                                apply(lambda x: freq_product_color(x, df))

In [262]:
df_unique = df_colour.drop_duplicates(subset=['product'])

In [269]:
for elem in list(df_unique[df_unique.colour.isna()]['product']):
    print(elem)

BCAA : Апельсин
BCAA : Лимон
Basketball ball Wilson NBA Authentic Series Outdoor
Bike computer Stern CBC-5 5 functions black
Demix fitness gloves red
Demix 袜子，3 双黑色
FILA socks, 3 pairs black/white
FILA 蓝色男士泳裤
Futsal ball Mikasa FSC-62
Giày nam Demix Exponenta 3 Fg đỏ/trắng
Giày thể thao PUMA Twitch Runner dành cho nữ Màu hồng / trắng / đen tươi
Gladiator BIG Sup 17.0
Gladiator DUO 15.2
Gladiator ELITE 11.2
Gladiator ELITE 11.6
Gladiator ELITE 12.6LT
Gladiator ELITE 12.6S
Gladiator ELITE 14.0S
Gladiator ELITE KD 11.6R
Gladiator ORIGIN 10.6SC
Gladiator ORIGIN 12.6S SC
Gladiator ORIGIN 12.6T SC
Gladiator PRO 11.2
Gladiator PRO 11.4
Gladiator PRO 12.6T
Gladiator WIND 10.7
Gladiator WIND 11.6
Joss Joy 深蓝色拖鞋
Kлюшка CCM JetSpeed FT5 SR
Low shoes for men Caterpillar Downlink Canvas blue/white
Low shoes for men Outventure Lighttrack Pro black color
Men's windbreaker Outventure dark blue color
Nike Value Cotton Crew socks, 3 pairs
PUMA 4Keeps sports bra top lavender color
Quần tây nam FILA đen
S

In [ ]:
# Обработаем датасет в мультипроцессорном режиме с замером времени
# (удалим из текста html-теги, неалфавитные символы, переведём в нижний регистр)

start_time = dt.datetime.now()
print('ВРЕМЯ СТАРТА РАСЧЁТА:', start_time, '\n')

# Запарралелим расчёты между доступными ядрами процессора:

# 1. рассчитываем количество ядер процесса, которые задействуем для вычислений
#    (уменьшаем на 1 ядро, которое оставляем на системные потребности)
n_cpu = multiprocessing.cpu_count() - 1
print('Количество доступных процессоров:', n_cpu)

# 2. Расчитаем размер обрабатываемого парралельно подмассива
val_sub = int(len(df)/n_cpu)
print('Размеры подмассивов:', val_sub, '\n')

# 3. Создаём менеджера парралельных процессов
#    и его пространство имён для обмена данными между процессами
if __name__ == '__main__':
    with Manager() as mng:
        ns = mng.Namespace()
        ns.dfs = mng.list()
        # 4. Делим датасет на составляющие,
        #    которые будут рассчитываться в парралельных процессах
        for i in range(n_cpu - 1):
            ns.dfs.append(df[val_sub*i:val_sub*(i+1)])
        ns.dfs.append(df[(n_cpu-1)*val_sub:])

        # 5. Запустим пул парралельных процессов вычислений
        #    и объединим результат в единый датафрейм
        with Pool(n_cpu) as p:
            df = pd.concat(list(p.map(lem_text, ns.dfs)), axis=0)

finish_time = dt.datetime.now()
print('\n\nВРЕМЯ ОКОНЧАНИЯ РАСЧЁТА:', finish_time)
print('ВРЕМЯ ВЫПОЛНЕНИЯ:', finish_time - start_time, '\n' * 2)

In [257]:
list(df_colour_unique['product'])

['8 батареек GP, АА, 8 шт.',
 'Apple Watch Series 8, 41 мм, корпус из алюминия',
 'Apple Watch Series 8, 41 мм, корпус из алюминия, спортивный ремешок в комплекте',
 'BCAA : Апельсин',
 'BCAA : Лимон',
 'Basketball ball Wilson NBA Authentic Series Outdoor',
 'Bike computer Stern CBC-5 5 functions black',
 'Cамокат Nitro Circus Ryan Williams 500, 120 мм',
 'Cамокат Nitro Circus Ryan Williams 560, 120 мм',
 'Cамокат Nitro Circus Ryan Williams Jr. 100 мм',
 'Cамокат Nitro Circus Ryan Williams, 110 мм',
 'Cдвоенный массажный мяч Trigger Point MB2',
 'Cмазка силиконовая для беговых дорожек Daytona',
 'Cноуборд Salomon Sickstick',
 'Cуппорт колена LP',
 'Demix fitness gloves red',
 'Demix 袜子，3 双黑色',
 'FILA socks, 3 pairs black/white',
 'FILA 蓝色男士泳裤',
 'Futsal ball Mikasa FSC-62',
 'Giày nam Demix Exponenta 3 Fg đỏ/trắng',
 'Giày thể thao PUMA Twitch Runner dành cho nữ Màu hồng / trắng / đen tươi',
 'Gladiator BIG Sup 17.0',
 'Gladiator DUO 15.2',
 'Gladiator ELITE 11.2',
 'Gladiator ELITE 11

In [82]:
dfs['purchases'][dfs['purchases']['product'] == "蛋白质棒 Forsio ''Orange Rough'' 50 g 多色"]

,id,product,colour,cost,product_sex,base_sale,dt
757471,171611,蛋白质棒 Forsio ''Orange Rough'' 50 g 多色,None,83,NaN,0,11


In [46]:
# Посчитаем количество уникальных продуктов
dfs['purchases'].nunique()['product']

23145

In [74]:
# Выберем гендерную принадлежность потенциальных покупателей продукта
# в тех продуктах где она заполнена
df_product_sex = dfs['purchases'][dfs['purchases'].product_sex.isna()].\
                 groupby('product').\
                 agg({'product_sex': 'max'})
df_product_sex

,product_sex
product,
"8 батареек GP, АА, 8 шт.",NaN
"Apple Watch Series 8, 41 мм, корпус из алюминия",NaN
"Apple Watch Series 8, 41 мм, корпус из алюминия, спортивный ремешок в комплекте",NaN
BCAA : Апельсин,NaN
BCAA : Лимон,NaN
...,...
Ящик Plano 6202-06,NaN
Ящик Plano 6203-06,NaN
男士软壳夹克Arc'teryx Gamma 深蓝色,NaN


In [40]:
df_product_sex[df_product_sex['product_sex'] != 1]

,product_sex,cost
product,,
Maxler VitaWomen витамины для женщин. 90 таблеток,4,4
Анорак двусторонний мужской Termit,17,17
Анорак женский 686 Upton Insulated,6,6
Анорак женский ARC'TERYX Alpha SL,13,13
Анорак женский ARC'TERYX Venda,12,12
...,...,...
Юбка-шорты женская Salomon Sense,11,11
Юбка-шорты женская Termit,56,56
Юбка-шорты женская adidas,2,2


In [27]:
dfs['purchases'].nunique()

id             104989
product         23145
colour           1693
cost             2402
product_sex         2
base_sale           2
dt                 59
dtype: int64

In [115]:
# Колонки с пропусками
sorted(dfs['purchases']['product'].unique())

['8 батареек GP, АА, 8 шт.',
 'Apple Watch Series 8, 41 мм, корпус из алюминия',
 'Apple Watch Series 8, 41 мм, корпус из алюминия, спортивный ремешок в комплекте',
 'BCAA : Апельсин',
 'BCAA : Лимон',
 'Basketball ball Wilson NBA Authentic Series Outdoor',
 'Bike computer Stern CBC-5 5 functions black',
 'Cамокат Nitro Circus Ryan Williams 500, 120 мм',
 'Cамокат Nitro Circus Ryan Williams 560, 120 мм',
 'Cамокат Nitro Circus Ryan Williams Jr. 100 мм',
 'Cамокат Nitro Circus Ryan Williams, 110 мм',
 'Cдвоенный массажный мяч Trigger Point MB2',
 'Cмазка силиконовая для беговых дорожек Daytona',
 'Cноуборд Salomon Sickstick',
 'Cуппорт колена LP',
 'Demix fitness gloves red',
 'Demix 袜子，3 双黑色',
 'FILA socks, 3 pairs black/white',
 'FILA 蓝色男士泳裤',
 'Futsal ball Mikasa FSC-62',
 'Giày nam Demix Exponenta 3 Fg đỏ/trắng',
 'Giày thể thao PUMA Twitch Runner dành cho nữ Màu hồng / trắng / đen tươi',
 'Gladiator BIG Sup 17.0',
 'Gladiator DUO 15.2',
 'Gladiator ELITE 11.2',
 'Gladiator ELITE 11

## 1.3. Обогащение данных таблицы personal_data из дополнительного источника

In [70]:
# Прочитаем восстановленные данные
df_recovered  = pd.read_csv('data/result_data/personal_data.csv')
df_recovered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15748 entries, 0 to 15747
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         15748 non-null  int64 
 1   age        15748 non-null  int64 
 2   education  15748 non-null  object
 3   city       15748 non-null  int64 
 4   country    15748 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 615.3+ KB


In [71]:
# Получим восстановленные данные о клиентах
df_recovered.head(5)

,id,age,education,city,country
0,101492,32,среднее,1188,32
1,42690,20,высшее,1134,32
2,113001,27,среднее,1187,32
3,163654,49,высшее,1160,32
4,69523,44,среднее,1137,32


In [85]:
# Добавим восстановленные данные в общий датафрейм
dfs['personal_data_recovered'] = pd.concat([dfs['personal_data'], df_recovered], axis=0)
dfs['personal_data_recovered'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104989 entries, 0 to 15747
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id         104989 non-null  int64  
 1   gender     89241 non-null   float64
 2   age        104989 non-null  int64  
 3   education  104989 non-null  object 
 4   city       104989 non-null  int64  
 5   country    104989 non-null  int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 5.6+ MB


In [86]:
dfs['personal_data_recovered'].head(5)

,id,gender,age,education,city,country
0,0,0.0,36,среднее,1201,32
1,4,0.0,35,среднее,1134,32
2,6,1.0,52,среднее,1188,32
3,7,0.0,37,среднее,1198,32
4,9,0.0,48,среднее,1134,32


In [99]:
# Проверим соответсвие полуxенных таблиц продаж и данных о клиентах на совпадение уникальных id
coef_1 = len(set(dfs['personal_data_recovered'].id) - set(dfs['purchases'].id))
coef_2 = len(set(dfs['personal_data_recovered'].id) - set(dfs['personal_data_coeffs'].id))

if not coef_1:
    print('Таблица продаж по полю id соответствует таблице данных о клиентах')
else:
     print('Таблица продаж по полю id расходится с таблицей данных о клиентах')
if not coef_2:
    print('Таблица данных о клиентах по полю id соответствует таблице коэффициентов каждого клиента')
else:
     print('Таблица данных о клиентах по полю id расходится с таблицей коэффициентов каждого клиента')

Таблица продаж по полю id соответствует таблице данных о клиентах
Таблица данных о клиентах по полю id соответствует таблице коэффициентов каждого клиента


# 2. Бинарная классификация

# 3. Анализ проведённого АВ-тестирования

# 4. Кластерезация

# 5. Построение модели склорнности клиентов к покупкам